In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列

In [1]:
import pandas as pd

# 读入 CSV 文件
df = pd.read_csv('evaluate_choco_seg.csv')
# df = pd.read_csv('evaluate_seg.csv')

df = df.drop(columns=['pbid'])
df = df[df['ARG'] <= 100000]
# df = df[df['classcial'] <= 500]
# df = df[df['in_constraints_probs'] >= 99]
grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    'iteration_count':'mean',
    'classcial':'mean',
    'run_times':'mean',
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 'mean',
})

## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=["ARG", 'best_solution_probs', 'classcial', 'run_times','iteration_count'])
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

method_order = ['ChocoSolver', 'ChocoSegmentedSolver']
# method_order = ['HeaSolver', 'PenaltySolver', 'ChocoSolver', 'QtoSolver ', 'QtoSimplifySolver', 'QtoSimplifyDiscardSolver', 'QtoSimplifyDiscardSegmentedSolver']
# method_order = ['ChocoSolver', 'QtoSolver', 'QtoSimplifySolver', 'QtoSimplifyDiscardSolver']
# method_order = ['ChocoSolver']
# method_order = ['QtoSimplifyDiscardSegmentedSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['classcial', "ARG",'best_solution_probs','run_times','iteration_count'], method_order]))

pivot_df

classcial                              ARG  \
                           ChocoSolver ChocoSegmentedSolver ChocoSolver   
pkid variables constraints                                                
0    6         3              1.282869            51.863106    0.087179   
1    15        8             10.635557           205.742584    0.415217   
2    21        12            22.634325           312.689334    0.336269   
3    28        15            55.515775           549.557085    0.503298   
4    12        6              5.164488           166.508942    0.228525   
5    15        9              4.333254           116.915192    0.623993   
6    20        8             43.542070          1548.446475    1.089941   
7    24        12            29.328615           745.494176    1.543578   
8    8         6              3.187495            94.503570    0.367541   
9    15        8              6.995467           148.769588    0.966064   
10   18        9             31.695708           314.077946    1.543593   
11   21        10            48.305431           359.829572    3.162708   
12   7         4              2.068468            73.646155    0.060758   
13   10        5              3.917582           107.741002    0.418355   
14   14        6             15.658534           215.393778    0.549831   
15   18        7             21.192509           277.842605    0.778376   
16   9         4              7.889739           123.962101    2.660405   
17   12        5             28.584189           340.577688    8.538143   
18   16        6            102.273511          1108.696063   23.733328   
19   20        7            284.354118          2033.010544   31.532932   

                                                best_solution_probs  \
                           ChocoSegmentedSolver         ChocoSolver   
pkid variables constraints                                            
0    6         3                       0.102948           90.006836   
1    15        8                       0.471778           32.960938   
2    21        12                      0.307022           26.099609   
3    28        15                      0.448265           18.539062   
4    12        6                       0.309435           53.154297   
5    15        9                       0.614845           34.772461   
6    20        8                       1.268387           12.521484   
7    24        12                      1.604872            5.716797   
8    8         6                       0.908905           82.798713   
9    15        8                       1.012318           27.077148   
10   18        9                       1.361179           15.224609   
11   21        10                      3.106842            6.065762   
12   7         4                       0.112859           90.740234   
13   10        5                       0.416304           51.999023   
14   14        6                       0.540679           31.421875   
15   18        7                       0.756251           11.348633   
16   9         4                       4.817925           65.566406   
17   12        5                      16.031136           37.963867   
18   16        6                      32.648369           17.317383   
19   20        7                      51.054148           12.575195   

                                                     run_times  \
                           ChocoSegmentedSolver    ChocoSolver   
pkid variables constraints                                       
0    6         3                      81.448242  304179.200000   
1    15        8                      25.971680  305131.520000   
2    21        12                     25.807617  305274.880000   
3    28        15                     14.856445  304537.600000   
4    12        6                      43.219727  303124.480000   
5    15        9                      34.011719  300554.240000   
6    20        8                       5.954102  301547.520000   
7    24  

In [2]:
# 1.597037+8.880065+19.523182+49.657212=79.65749600000001
# 0.848695+9.357230+27.663566+140.267882=178.13737299999997
178.13737299999997 / 79.65749600000001

2.2362913968573648

In [3]:
# # 假设“variables”和“constraints”列作为多重索引列的一部分
# # 使用 .loc 选择多重索引列中含“variables”和“constraints”相关的列
# selected_columns = pivot_df.loc[:, pivot_df.columns.get_level_values(0).isin(['variables', 'constraints'])]

# selected_columns


In [4]:
# 选取 'depth' 下的所有列，并剔除 'QtoSolver ' 和 'QtoSimplifySolver'
df_1 = pivot_df['ARG'].drop(columns=['QtoSolver ', 'QtoSimplifySolver'], errors='ignore')

# 选取 'culled_depth_per_param' 下的 'QtoSimplifyDiscardSolver' 列
df_2 = pivot_df['classcial'].drop(columns=['QtoSolver ', 'QtoSimplifySolver'], errors='ignore')

# 合并处理后的 'depth' 列与 'QtoSimplifyDiscardSolver' 列，并指定后缀避免冲突
culled_depth_df = df_1.join(df_2, lsuffix='_1', rsuffix='_2')

# 按行遍历输出值，每个值用 & 分隔
for row in culled_depth_df.itertuples(index=False):
    formatted_row = [f"{round(x, 3):6.03f}" for x in row]
    print(" & ".join(formatted_row))


 0.087 &  0.103 &  1.283 & 51.863
 0.415 &  0.472 & 10.636 & 205.743
 0.336 &  0.307 & 22.634 & 312.689
 0.503 &  0.448 & 55.516 & 549.557
 0.229 &  0.309 &  5.164 & 166.509
 0.624 &  0.615 &  4.333 & 116.915
 1.090 &  1.268 & 43.542 & 1548.446
 1.544 &  1.605 & 29.329 & 745.494
 0.368 &  0.909 &  3.187 & 94.504
 0.966 &  1.012 &  6.995 & 148.770
 1.544 &  1.361 & 31.696 & 314.078
 3.163 &  3.107 & 48.305 & 359.830
 0.061 &  0.113 &  2.068 & 73.646
 0.418 &  0.416 &  3.918 & 107.741
 0.550 &  0.541 & 15.659 & 215.394
 0.778 &  0.756 & 21.193 & 277.843
 2.660 &  4.818 &  7.890 & 123.962
 8.538 & 16.031 & 28.584 & 340.578
23.733 & 32.648 & 102.274 & 1108.696
31.533 & 51.054 & 284.354 & 2033.011


In [5]:
# # 从 pivot_df 中提取 variables 和 constraints 列
# variables_constraints_df = pivot_df.index.to_frame(index=False)[['variables', 'constraints']]
# variables_constraints_df

In [6]:
# 提取所需列
best_solution_probs_Qto_discard = pivot_df['best_solution_probs']['QtoSimplifyDiscardSolver']
best_solution_probs_choco = pivot_df['best_solution_probs']['ChocoSolver']
arg_Qto_discard = pivot_df['ARG']['QtoSimplifyDiscardSolver']
arg_choco = pivot_df['ARG']['ChocoSolver']

# 方法1：每行相除取平均
# best_solution_probs 下 QtoSimplifyDiscardSolver / ChocoSolver 的倍数
row_wise_ratio_best_solution_probs = (best_solution_probs_Qto_discard / best_solution_probs_choco).mean()

# ARG 下 ChocoSolver / QtoSimplifyDiscardSolver 的倍数
row_wise_ratio_arg = (arg_choco / arg_Qto_discard).mean()

# 方法2：整列取平均后相除
# best_solution_probs 下 QtoSimplifyDiscardSolver / ChocoSolver 的整列平均的倍数
col_avg_ratio_best_solution_probs = best_solution_probs_Qto_discard.mean() / best_solution_probs_choco.mean()

# ARG 下 ChocoSolver / QtoSimplifyDiscardSolver 的整列平均的倍数
col_avg_ratio_arg = arg_choco.mean() / arg_Qto_discard.mean()

# 输出结果
print(f"行平均 - QtoSimplifyDiscardSolver / ChocoSolver (best_solution_probs): {row_wise_ratio_best_solution_probs:.2f}")
print(f"行平均 - ChocoSolver / QtoSimplifyDiscardSolver (ARG): {row_wise_ratio_arg:.2f}")
print(f"列平均 - QtoSimplifyDiscardSolver / ChocoSolver (best_solution_probs): {col_avg_ratio_best_solution_probs:.2f}")
print(f"列平均 - ChocoSolver / QtoSimplifyDiscardSolver (ARG): {col_avg_ratio_arg:.2f}")


KeyError: 'QtoSimplifyDiscardSolver'

In [ ]:

# # 读入 CSV 文件
# df = pd.read_csv('evaluate_TSP.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })

# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [ ]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eva_SCP.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })

# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [ ]:
# 定义格式化函数
import numpy as np
def format_value(val):
    if np.isnan(val):  # 如果是NaN值则返回空字符
        return "NaN"
    elif val < 100:  # 如果值小于100，保留3位有效数字
        return f"{val:.2f}"
    else:  # 如果值大于等于100，不保留小数点
        return f"{val:.0f}"

# 按行遍历输出，每个值用 & 分隔
for row in pivot_df.itertuples(index=False):
    formatted_row = [format_value(x) for x in row]
    print(" & ".join(formatted_row))

1.60 & 0.85 & 0.46 & 0.08 & 0.01 & 0.00 & 90.17 & 96.08 & 99.79 & 304876 & 285788 & 73615 & 301 & 301 & 185
8.88 & 9.36 & 10.44 & 0.44 & 0.13 & 0.12 & 33.44 & 71.26 & 66.42 & 305428 & 308224 & 308224 & 301 & 301 & 301
19.52 & 27.66 & 32.51 & 0.35 & 0.20 & 0.13 & 26.94 & 51.21 & 61.48 & 304937 & 308224 & 308224 & 301 & 301 & 301
49.66 & 140 & 249 & 0.51 & 0.65 & 0.66 & 18.70 & 13.97 & 14.40 & 306545 & 308224 & 308224 & 301 & 301 & 301
5.57 & 3.28 & 3.33 & 0.24 & 0.18 & 0.26 & 51.58 & 55.89 & 47.10 & 303360 & 308224 & 247798 & 301 & 301 & 294
4.35 & 4.03 & 2.60 & 0.61 & 0.23 & 0.42 & 35.28 & 62.59 & 37.93 & 301568 & 308224 & 222659 & 301 & 301 & 298
41.51 & 19.45 & 18.54 & 1.12 & 0.40 & 0.43 & 10.43 & 40.02 & 36.42 & 302592 & 308224 & 308224 & 301 & 301 & 301
23.99 & 25.06 & 16.29 & 1.58 & 0.76 & 0.91 & 5.80 & 33.69 & 26.67 & 302828 & 308224 & 307364 & 301 & 301 & 301
3.22 & 1.71 & 1.55 & 0.30 & 0.02 & 0.40 & 83.92 & 97.33 & 84.42 & 304610 & 287650 & 155182 & 301 & 301 & 245
6.71 & 6.14 

In [ ]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eva_old.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })
# grouped_df
# # 假设grouped_df是一个Pandas DataFrame
# grouped_df = grouped_df[((grouped_df['layers'] == 1) & (grouped_df['method'] == 'commute')) | ((grouped_df['layers'] == 7) & (grouped_df['method'] != 'commute'))]
# grouped_df = grouped_df.drop(columns=['layers'])
# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])




# method_order = ['penalty', 'cyclic', 'HEA', 'commute']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [ ]:
pivot_df['ARG']

,,,ChocoSolver,QtoSimplifySolver,QtoSimplifyDiscardSolver
pkid,variables,constraints,,,
0,6,3,0.082210,0.009911,0.001090
1,15,8,0.437341,0.130795,0.122484
2,21,12,0.345957,0.196412,0.134328
3,28,15,0.514408,0.653753,0.660056
4,12,6,0.240343,0.178111,0.262196
5,15,9,0.607625,0.231107,0.418823
6,20,8,1.120499,0.404076,0.428920
7,24,12,1.577100,0.760320,0.908460
8,8,6,0.296257,0.018257,0.400939


In [ ]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eval_eliminate copy 2.csv')

# df = df.drop(columns=['pbid','layers', 'variables', 'constraints'])

# gr    ouped_df = df.groupby(['pkid',  'method'], as_index=False).agg({
#     "successrate": 'mean',
# })
# grouped_df
# # 假设grouped_df是一个Pandas DataFrame
# # grouped_df = grouped_df[((grouped_df['layers'] == 1) & (grouped_df['method'] == 'commute')) | ((grouped_df['layers'] == 7) & (grouped_df['method'] != 'commute'))]
# # grouped_df = grouped_df.drop(columns=['layers'])
# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])




# method_order = ['penalty', 'cyclic', 'HEA', 'commute']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df